# Transfer of Airport Movement Data

Notebook addressing the creation of an Excel file report for the "Transfer of Airport Movement Data" on Lommis Airfield.

Template used in accordance with the one provided by the Federal Office of Civil Aviation (BAZL): https://www.bazl.admin.ch/bazl/de/home/themen/geoinformation_statistik/statistik/statistische_datenlieferungen.html

In [1]:
## IMPORT LIBRARIES ##
import os
import lommis_func
import warnings
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from datetime import datetime, timedelta
from traffic.core import Traffic, Flight

from traffic.data import airports, eurofirs, opensky
warnings.simplefilter("ignore")

In [23]:
# Year and Month of the fetching data
year = 2025
month = 3

In [197]:
## FETCH DATA FROM LOMMIS AIRFIELD ##
import time 

# Define the start date
start_date = datetime.strptime(f"{year}-{month}-01 00:00", "%Y-%m-%d %H:%M")
 
# Number of iterations
num_iterations = 31

# Define the parquet directory path
folder_name = f"{month:02d}{year}"
parquet_dir_path = os.path.join("Flights/Others/", folder_name)

os.makedirs(parquet_dir_path, exist_ok=True)
# Delete all contents of the parquet directory if any exists
# if os.path.exists(parquet_dir_path):
#     shutil.rmtree(parquet_dir_path)
#     os.makedirs(parquet_dir_path)
#     print(f"Deleted all contents in the directory: {parquet_dir_path}")

for i in range(num_iterations):
    # Generate stop date
    stop_date = start_date + timedelta(days=1)
 
    # Format both dates as strings in the desired format
    start_str = start_date.strftime("%Y-%m-%d %H:%M")
    stop_str = stop_date.strftime("%Y-%m-%d %H:%M")
    print(f"Fetching data for Start: {start_str}, Stop: {stop_str}\n")
    
    try:
        # fetch data
        trajs = opensky.history(
            start = start_str,
            stop = stop_str,
            # aircraft : HB-TXN, HB-KOJ, HB-SGW, HB-SGR, D-MDUE
            icao24='486740',
            #callsign= ["HBKOJ", "HBSGR"],
            #bounds = airports['LSZT'].shape.buffer(0.1).bounds,
            # airport = "LSZT",
            selected_columns=(),
        )
    except Exception as e:
        print(f"Error fetching data for {start_str} to {stop_str}: {e}")
        trajs = None
    
    if trajs is not None:
        file_date = start_date.strftime("%Y-%m-%d")
        #trajs.to_parquet(os.path.join(parquet_dir_path, f"{file_date}.parquet"))

        # Filter for aligned over runway 24
        # aligned_flights = []
        # for flight in trajs:
        #     min_alt = flight.data.altitude.min()

        #     if pd.notna(min_alt) and min_alt < airports["LSZT"].altitude + 300:
        #         seg = lommis_func.aligned_over_runway(flight, airports["LSZT"], '24', scale=1.5, debug=False)
                

        #         if seg is not None and len(seg) >= 1:
        #             #landed    = flight.landing_at(airports["LSZT"])
        #             #took_off  = flight.takeoff_from(airports["LSZT"])

        #             #if landed or took_off:
        #             aligned_flights.append(flight)

        # Save aligned only if we found any
        # if aligned_flights:
        t_lszt = Traffic.from_flights(trajs)
        t_lszt.to_parquet(os.path.join(parquet_dir_path, f"{file_date}.parquet"))
        print(f"Saving flight {file_date} with {len(t_lszt)} trajectories...\n")
        time.sleep(15)

    # Update date
    start_date += timedelta(days=1)

Fetching data for Start: 2025-03-01 00:00, Stop: 2025-03-02 00:00



KeyboardInterrupt: 

In [24]:
## RETRIEVE FETCHED DATA FROM FOLDER ##
flights = []
folder_name = f"{month:02d}{year}"
folder_path = os.path.join("Flights/", folder_name)
for filename in os.listdir(folder_path):

    if filename.endswith(".parquet"):
        file_path = os.path.join(folder_path, filename)
        flight = Flight.from_file(file_path)
        flights.append(flight)

traff_set = Traffic.from_flights(flights)

#convert flights Timestamp to 'datetime64'
traff_set.data['timestamp'] = traff_set.data['timestamp'].dt.tz_convert(None).astype('datetime64[ns]')
traff_set.data['timestamp'] = traff_set.data['timestamp'].dt.tz_localize('UTC')

In [25]:
traff_set

,,count
icao24,callsign,
4b3037,HBSGT,78355
4b303a,HBSGW,57790
4b1b97,HBKLR,54183
4b0c84,HBETG,48608
4b1b86,HBKLA,36346
4b1b00,HBKFW,33501
3ff741,DMDUE,31550
4b1b89,HBKLD,27200
4b4184,HBYVC,19465


In [16]:
## FILTERING FLIGHTS ##
from traffic.core.mixins import PointMixin
_, _, center = lommis_func.retrieve_runway_information(airports["LSZT"], "06")

filtered_flights = []
numeric_columns = ["latitude", "longitude", "track", "altitude", "groundspeed", "vertical_rate"]

for flight in traff_set:
    if flight.data[numeric_columns].isna().all().all():
        continue
    
    flight.data[numeric_columns] = flight.data[numeric_columns].ffill().bfill()

    if any((flight.data[numeric_columns] == 0).all()):
        continue
    
    if flight.data.dropna(subset=numeric_columns).empty:
        continue

    # dist = np.array(flight.distance(center).data['distance'])
    # if np.min(dist) > 1:  # if minimum distance is greater than 1 NM (not even close to airport)
    #     continue

    if not flight.data.index.is_unique:
        flight.data = flight.data.reset_index(drop=True)
    
    filtered_flights.append(flight)

traff_set_filtered = Traffic.from_flights(filtered_flights)
print(f"Total flights kept: {len(traff_set_filtered)}")

Total flights kept: 191


In [17]:
import os

# Filename and path
filename = f"{year}-{month:02d}-flights.parquet"
file_path = os.path.join("Statistics/Others/", filename)

# Save
traff_set_filtered.to_parquet(file_path)
print(f"Filtered flight data saved to: {file_path}")

Filtered flight data saved to: Statistics/Others/2025-03-flights.parquet


In [18]:
## ANALYZE FILTERED FLIGHTS AND CREATE DATA LIST ##
flight_data = []
for flight in traff_set_filtered:
    day = flight.data.timestamp.iloc[0].strftime("%d")
    
    lommis_func.analyze_flight(flight, airports["LSZT"], flight_data, [day, month, year], debug = False)

# sort columns by day and time
flight_data.sort(
    key=lambda row: (
        int(row[1]),
        datetime.strptime(row[4], "%H%M")
    )
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
## LOAD EXCEL TEMPLATE AND FILL IT OUT WITH NEW DATA ##
file_path = "Excel/template/template.xlsx"
wb = load_workbook(file_path)

ws_title = wb["TITLE"]

for row in ws_title.iter_rows():
    for cell in row:
        if cell.value == "Year":
            ws_title.cell(row=cell.row, column=cell.column + 2, value=year)
        elif cell.value == "Period":
            ws_title.cell(row=cell.row, column=cell.column + 2, value=month)

ws_data = wb["DATA"]

center_alignment = Alignment(horizontal="center", vertical="center")
for row in flight_data:
    ws_data.append(row)
    new_row_idx = ws_data.max_row
    for col_idx in range(1, len(row) + 1):
        cell = ws_data.cell(row=new_row_idx, column=col_idx)
        cell.alignment = center_alignment

new_filename = f"ARP_LSZT_{month:02d}{year}.xlsx"
new_file_path = os.path.join("Excel/", new_filename)

wb.save(new_file_path)
print(f"Modified Excel file saved as: {new_filename}")

c:\Users\fusg\Documents\Lommis_App\.venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Modified Excel file saved as: ARP_LSZT_032025.xlsx
